## Load latest headlines

In [1]:
# ========== Bootstrap: ensure required Python packages are present =========
import importlib, subprocess, sys
from typing import Optional

def _ensure(pkg_name: str, import_name: Optional[str] = None):
    try:
        importlib.import_module(import_name or pkg_name)
    except ModuleNotFoundError:
        subprocess.check_call([sys.executable, '-m', 'pip', 'install', pkg_name])
    finally:
        globals()[import_name or pkg_name] = importlib.import_module(import_name or pkg_name)

_ensure('pandas')
print('All dependencies ready.\n')


All dependencies ready.



In [2]:
from pathlib import Path
archive = Path('archive')
archive.mkdir(exist_ok=True)
import pandas as pd
latest = pd.read_csv('../headlines/latest.csv')
latest.head()

,pubdate,source,title,link
0,2025-12-21-16-51-28 +0000,nyt,Blanche Says Mentions of Trump in Epstein File...,https://www.nytimes.com/2025/12/21/us/politics...
1,2025-12-21-16-28-56 +0000,nypost,Bernie Sanders blasted for tanking bill to hel...,https://nypost.com/2025/12/21/us-news/bernie-s...
2,2025-12-21-16-17-01 +0000,wapo,U.S. tariffs take a bite out of Germany’s icon...,https://www.washingtonpost.com/world/2025/12/2...
3,2025-12-21-16-08-05 +0000,nypost,Russia claims talks on US peace plan for Ukrai...,https://nypost.com/2025/12/21/world-news/russi...
4,2025-12-21-15-56-08 +0000,nyt,"‘It’s Just Us’: The Firefighter, His Son and a...",https://www.nytimes.com/2025/12/21/us/wildfire...


## Calculate word frequencies

In [3]:
import re
from collections import Counter
from datetime import datetime

with open('exclude.txt') as f:
    stop_words = set(w.strip() for w in f if w.strip())
words = re.findall(r'[A-Za-z]+', ' '.join(latest['title']).lower())
filtered = [w for w in words if w not in stop_words and len(w) > 1]
counts = Counter(filtered)
score_df = (
    pd.DataFrame(counts.items(), columns=['word','score'])
    .sort_values('score', ascending=False)
)
score_df[['score','word']].to_csv('scores.csv', index=False)
timestamp = datetime.utcnow().strftime('%Y-%m-%d-%H-00-00')
score_df[['score','word']].to_csv(archive / f'scores-{timestamp}.csv', index=False)
score_df.head()


/tmp/ipykernel_2388/2220774760.py:15: DeprecationWarning: datetime.datetime.utcnow() is deprecated and scheduled for removal in a future version. Use timezone-aware objects to represent datetimes in UTC: datetime.datetime.now(datetime.UTC).
  timestamp = datetime.utcnow().strftime('%Y-%m-%d-%H-00-00')


,word,score
2,trump,17
3,epstein,11
170,bondi,9
56,oil,8
4,files,8


## Rank headlines by score

In [4]:
word_scores = dict(score_df[['word','score']].values)
latest['score'] = latest['title'].apply(
    lambda t: sum(
        word_scores.get(w.lower(), 0)
        for w in re.findall(r'[A-Za-z]+', t)
        if len(w) > 1
    )
)
ranked = latest.sort_values('score', ascending=False)
ranked[['score','pubdate','source','title','link']].to_csv('rank.csv', index=False)
ranked[['score','pubdate','source','title','link']].to_csv(archive / f'rank-{timestamp}.csv', index=False)
ranked.head()


,pubdate,source,title,link,score
107,2025-12-21-01-59-17 +0000,wapo,Coast Guard boards oil tanker leaving Venezuel...,https://www.washingtonpost.com/national-securi...,55
126,2025-12-20-23-45-01 +0000,nyt,Epstein Files Photos Disappear From Government...,https://www.nytimes.com/2025/12/20/us/politics...,49
140,2025-12-20-21-46-48 +0000,nypost,Russian envoy KO’s Trump administration propos...,https://nypost.com/2025/12/20/world-news/russi...,46
66,2025-12-21-10-00-00 +0000,wsj,"Before Charlie Kirk was fatally shot, he was r...",https://www.wsj.com/politics/policy/charlie-ki...,45
136,2025-12-20-22-07-00 +0000,wsj,Industry titans and Trump allies are weighing ...,https://www.wsj.com/politics/policy/the-fight-...,41


## Select top headlines

In [5]:
with open('exclude_sources.txt') as f:
    excluded_sources = {
        line.strip()
        for line in f
        if line.strip() and not line.strip().startswith('#')
    }

top_count = 10
top_rows = []
working = word_scores.copy()
remaining = latest.copy()

while len(top_rows) < top_count and not remaining.empty:
    scored_remaining = remaining.assign(
        score=remaining['title'].apply(
            lambda t: sum(
                working.get(w.lower(), 0)
                for w in re.findall(r'[A-Za-z]+', t)
                if len(w) > 1
            )
        )
    )
    ranked_loop = scored_remaining.sort_values('score', ascending=False)

    if ranked_loop.empty:
        break

    top_story = ranked_loop.iloc[0]
    remaining = remaining.drop(top_story.name)

    if top_story['source'] in excluded_sources:
        continue

    words = set(re.findall(r'[A-Za-z]+', top_story['title'].lower()))
    for word in words:
        working.pop(word, None)

    top_rows.append(top_story[['score', 'pubdate', 'source', 'title', 'link']])

top_df = pd.DataFrame(top_rows)
top_df.to_csv('top.csv', index=False)
top_df.to_csv(archive / f'top-{timestamp}.csv', index=False)
top_df


,score,pubdate,source,title,link
107,55,2025-12-21-01-59-17 +0000,wapo,Coast Guard boards oil tanker leaving Venezuel...,https://www.washingtonpost.com/national-securi...
66,45,2025-12-21-10-00-00 +0000,wsj,"Before Charlie Kirk was fatally shot, he was r...",https://www.wsj.com/politics/policy/charlie-ki...
38,35,2025-12-21-11-01-08 +0000,nypost,Defiant sister of slain Bondi Beach rabbi — wh...,https://nypost.com/2025/12/21/world-news/defia...
100,33,2025-12-21-03-07-25 +0000,nypost,More than a dozen newly-released Jeffrey Epste...,https://nypost.com/2025/12/20/us-news/nearly-2...
67,31,2025-12-21-09-52-21 +0000,nypost,"Australian PM Albanese booed, heckled as he si...",https://nypost.com/2025/12/21/world-news/austr...
140,29,2025-12-20-21-46-48 +0000,nypost,Russian envoy KO’s Trump administration propos...,https://nypost.com/2025/12/20/world-news/russi...
20,27,2025-12-21-14-13-42 +0000,nypost,117th Newport Beach Christmas Boat Parade ligh...,https://nypost.com/2025/12/21/us-news/117th-ne...
46,27,2025-12-21-11-00-00 +0000,wsj,Cleveland Fed President Beth Hammack said she ...,https://www.wsj.com/economy/central-banking/fe...
93,26,2025-12-21-04-30-13 +0000,nypost,"San Francisco power outage left 130,000 in the...",https://nypost.com/2025/12/20/us-news/san-fran...
165,23,2025-12-20-17-07-47 +0000,nypost,Jordan reveals air force joined US strikes on ...,https://nypost.com/2025/12/20/world-news/jorda...


## Create JSON version of top headlines

In [6]:
import pandas as pd
pd.read_csv('top.csv').to_json('top.json', orient='records', indent=2)
